In [1]:
%matplotlib widget
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.optimize import curve_fit
import json
import uproot
from Utils import *
from matplotlib import ticker, cm
import succolib as sb
from matplotlib.ticker import LogFormatter 
import succolib as sl
from scipy.integrate import quad

In [2]:
config_file = r"./config.json"

with open(config_file, "r") as f:
            dizi = json.load(f)
dizi

{'RunAllignment': 720374,
 'd_12': 921,
 'd_23': 91,
 'd_2crist': 43,
 'd_1crist': 964,
 'd_13': 1012,
 'd_tele1BC2': 1580,
 'data_dir': '/eos/project/i/insulab-como/testBeam/TB_2023_08_H8_SELDOM/HDF5',
 'offset_x2': 0.07806817598347049,
 'offset_y2': -0.3215607824343533,
 'offset_x3': 0.11587483304062503,
 'offset_y3': -0.5105463416185236,
 'offset_x4': -0.038727162032854784,
 'offset_y4': 4.21592233820313,
 'div_12x': 3.9158902242421664e-05,
 'div_12y': 2.856314570199142e-05,
 'div_13x': 3.8990252314782414e-05,
 'div_13y': 2.7872540269441723e-05,
 'div_BC1x': 3.758456156762604e-05,
 'div_BC1y': 2.7299236033839342e-05}

In [3]:
#720227 con run di allineamento 720218
#720255 

runs=[720379] 
Brucoplot = False
div_ang_func_x = False
data_dir = dizi["data_dir"]
pos =[]
infos=[]
for run in runs:
    data_path = f'{data_dir}/run{run}.h5'

    with h5py.File(data_path, 'r', libver='latest', swmr=True) as hf:
        #print(hf.keys())
        hf["xpos"].shape
        keys = list(hf.keys())
        #for k in hf.keys():
        #    comand = f'{k} = np.array(hf["{k}"])'
            # print(comand)
          #  exec(comand)
        pos.append(np.array(hf['xpos']))
        infos.append(np.array(hf['xinfo']))

#print(np.shape(pos))
#print(np.shape(infos))
        
xpos = np.concatenate(pos,axis=0)
xinfo = np.concatenate(infos,axis=0)

#print(np.shape(xpos))
#print(np.shape(xinfo))

logic = (xpos > 0) & (xpos < 10)
logic2 = logic.all(axis = 1)
xpos = xpos[logic2]
xinfo = xinfo[logic2]


my_cmap= copy.copy(mpl.cm.magma)
my_cmap.set_bad(my_cmap(0))


In [4]:
xpos[:,2] -= dizi['offset_x2']
xpos[:,3] -= dizi['offset_y2']
xpos[:,4] -= dizi['offset_x3']
xpos[:,5] -= dizi['offset_y3']
xpos[:,6] -= dizi['offset_x4']
xpos[:,7] -= dizi['offset_y4']
#########################################
x1, y1 = xpos[:,0], xpos[:,1]
x2, y2 = xpos[:,2], xpos[:,3]
x3, y3 = xpos[:,4], xpos[:,5]
x4, y4 = xpos[:,6], xpos[:,7] # BC1

rot = xinfo[:,0]

def proiettaDistZ(z):
    mx = (x2-x1)/dizi["d_12"]
    xProj = x1 + mx * z
    
    my = (y2-y1)/dizi["d_12"]
    yProj = y1 + my * z
    
    return (xProj, yProj)
    
x_cry, y_cry = proiettaDistZ(dizi["d_12"]+dizi["d_2crist"])



In [5]:
###Si potrebbe fare un plot sulle correlazioni delle variabili, x1, x2; y1, y2; ...

### Definizione delle variabili



In [6]:
# definizione di theta x in
theta_in_x = np.arctan((x2-x1)/dizi['d_12']) * 1e6 # urad
theta_in_y = np.arctan((y2-y1)/dizi['d_12']) * 1e6 # urad

# theta out calcolato tra tele3 e BC1 ---> PER ANGOLI PICCOLI
d_3BC1 = dizi["d_tele1BC2"]-dizi["d_13"]

theta_out_x = np.arctan((x4-x3)/d_3BC1) * 1e6 # urad
theta_out_y = np.arctan((y4-y3)/d_3BC1) * 1e6 # urad
thetain = [theta_in_x, theta_in_y]
thetaout = [theta_out_x, theta_out_y]
print(d_3BC1)


568


### Plot istogrammi $\theta_{in}$, $\theta_{out}$ 

In [7]:
thetaLbl = ["x","y","_","x","y","_"]#,"x","y","_"]
num_bins = 500
fig, ax = plt.subplots(2,3)
fig.set_size_inches(10,7)
fig.subplots_adjust(wspace=.4,hspace=.8)
ax=ax.flatten()
print(ax.shape)
titles = [fr"$\theta_X in$ ", fr"$\theta_Y in $","_",fr"$\theta_X out $ ", fr"$\theta_Y out $","_"]
titels2= [fr"$\theta_X out2 $ ", fr"$\theta_Y out2 $","_"]

fit_par=[]
for i in range(5):
    if i== 2 or i==5:
        continue
    if i>=0 and i<2 :
        h, bins = np.histogram(thetain[i], bins = 100, range =[-300,300])
        binc = bins[:-1] + (bins[1] - bins[0]) / 2
        p0 = (np.max(h), binc[np.argmax(h)], np.std(thetain[i]))
    elif i>2 and i<=4:
        h, bins = np.histogram(thetaout[i-3], bins = 500,  range =[-300,300])
        binc = bins[:-1] + (bins[1] - bins[0]) / 2
        p0 = (np.max(h), binc[np.argmax(h)], np.std(thetaout[i-3]))
        
    cond = h>0
    popt, pcov = curve_fit(myGauss, binc[cond], h[cond], sigma = np.sqrt(h[cond]), absolute_sigma = True, p0 = p0)
    fit_par.append(popt)

    ax[i].plot(binc, h, ds = "steps-mid", c = "green", lw = .7,
               label = fr"$\theta_{thetaLbl[i]}$")

    ax[i].plot(binc, myGauss(binc, *popt), ls = "--", c = "k", label = f"Fit ({popt[1]:.3f} $\pm$ {popt[2]:.3f} $\mu$ rad)")

    ax[i].grid()
    ax[i].set_title(titles[i])
    ax[i].set_ylabel("Entries", fontsize = 9)    
    ax[i].set_xlabel(r"$\theta$ [$\mu$rad]",fontsize = 9)

    ax[i].legend(fontsize = 7)

    print(f"La divergenza in {thetaLbl[i]} vale {popt[2]:.3f} \u03BCrad\n")


# PLOT hist2d
ax[2].hist2d(thetain[0], thetain[1], bins=100, range=[[-200,200],[-200,200]], cmap="inferno")
ax[5].hist2d(thetaout[0], thetaout[1], bins=100, range=[[-250,250],[-250,250]], cmap="inferno")


# Etichette degli assi
ax[2].grid()
ax[2].set_xlabel('thetax')
ax[2].set_ylabel('thetay')

ax[5].grid()
ax[5].set_xlabel('thetax')
ax[5].set_ylabel('thetay')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(6,)
La divergenza in x vale 37.476 μrad

La divergenza in y vale 26.930 μrad

La divergenza in x vale -46.215 μrad

La divergenza in y vale -34.624 μrad



#### Divergenza angolare in funzione della posizione in x del cristallo

In [8]:
    # Definizione delle posizioni
x_positions = xinfo[:,2]

x_val = (x_cry) #fattore 2 serve per terere conto del fatto che l'unità di misura delle posizioni del goniometro è di 2mm # +x_positions/2*0.1
DelfX = theta_out_x - theta_in_x
#DelfX2 = theta_out_x2 - theta_in_x  #tele3-BC2

In [9]:
 #if len(np.unique(xinfo[:,2]))>2 else False

if div_ang_func_x:

    fig, ax = plt.subplots()
    fig.set_size_inches(12, 5)

    # Istogramma dei dati 
    hh = ax.hist2d(x_val, DelfX, bins=[500, 200], range=[[np.min(x_val)+0.67,np.max(x_val)], [-150,150]], cmap=my_cmap,norm=mpl.colors.LogNorm())
    colorbar1 = fig.colorbar(hh[3], ax=ax)
    ax.axvline(x=0.963, c="white", ls="--")
    ax.axvline(x=1.157, c="white", ls="--")

    plt.show()


In [10]:
###DEFINIZIONE DEL TAGLIO LOGICO SUL CRISTALLO
#logi_cry=(x_val>5.68) & (x_val<7.8)
logi_cry=(x_val>0.928) & (x_val<1.121)
th_in_cut_low = fit_par[0][1] -10#2.5
th_in_cut_high = fit_par[0][1] +10#2.5
logi_th_in = (theta_in_x > th_in_cut_low) & (theta_in_x < th_in_cut_high)


In [11]:
div_ang_func_xv2 = False #div_ang_func_x

#questo è un plot delle posizioni delle particelle interagenti con il detector 2 proiettate sul cristallo, plottate per i d
steps= np.unique(xinfo[:,2])
if div_ang_func_xv2:    

    for step in steps:
        #Scelta dei limiti per theta in 
        
        fig, ax = plt.subplots(3,1, sharex = True)
        fig.set_size_inches(7, 5)

        #definisco le posizioni
        Dtheta = theta_out_x-theta_in_x
        logi_Dtheta = (Dtheta > 0) & (Dtheta < 500) #taglio logico sui Dtheta, per ridurre i bin 
        #Istogrammo i dati
        x_positions=xinfo[:,2]
        hh = ax[0].hist2d(x_cry[(logi_th_in) & (logi_Dtheta) & (x_positions == step)]*100, Dtheta[(logi_th_in) & (logi_Dtheta) & (x_positions == step)], bins = [50,2000])# , norm = mpl.colors.LogNorm(), cmap = my_cmap)
        # print(np.shape(hh[0]))

        ax[0].set_ylim(200, 400)


        binc = hh[1][:-1] + (hh[1][1] - hh[1][0]) / 2

        # print(np.max(hh[0]))
        tresh = 0.01*np.max(hh[0])
        binary = hh[0]>tresh
        # print(np.shape(np.sum(binary,axis=1)))
        # ax[2].imshow((binary).T,origin="lower")

        ax[1].plot(binc,np.sum(binary,axis=1))
        
        #Tentativo di fare il profile
        X_prof, Y_prof ,sigma_Y = sb.hist2dToProfile(hh, "mean")
        ax[2].errorbar(X_prof,Y_prof,sigma_Y)
        ax[2].grid()

        for a in ax:
            a.set_xlabel('x scan [mm]')
            a.set_ylabel('defl X[$\mu$rad]')
            #a.set_xlim(0.9,1.35)
            #a.axvline(0.895, c = 'hotpink')
            #a.axvline(x_cut_low, c = 'hotpink')
            #a.axvline(x_cut_high, c = 'hotpink')
        fig.colorbar(hh[3], ax = ax[0])
        #fig.savefig(f'./img/Div{i}.png')
        #fig.savefig(f'./pdf/Div{i}.pdf')
        #plt.close(fig)
    plt.show()

#### Bruco plot

In [12]:
#if len(np.unique(xinfo[:,0]))>1 else False

if Brucoplot:
    deflX = theta_out_x-theta_in_x
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 7)
    rot = xinfo[:,0]
    #fig.suptitle(f'Bruco Plot [{x_cut_low:.4f}_{x_cut_high:.4f} um]', fontsize = 20)
    bin_step = 2 #urad
    
    hh = ax.hist2d((rot+theta_in_x)[logi_cry],(deflX)[logi_cry], 
                    bins = [len(np.unique(xinfo[:,0]))*10, 300],
                    cmap = my_cmap,
                    range = [[min(rot)-300,max(rot)+300],[-1000,1000]])#,
                    #norm=mpl.colors.LogNorm())

    X_prof, Y_prof, sigma_Y = sb.hist2dToProfile(hh, "mean")
    ax.errorbar(X_prof, Y_prof, sigma_Y, color = "red", lw =0.2)
    #ax.grid()
    theta_c=50 #urad
    ax.set_xlabel('Theta X in [$\mu$rad]')
    ax.set_ylabel('Defl X [$\mu$rad]')
    
    fig.colorbar(hh[3], ax = ax)
    ax.axvline(x=-1126-theta_c/4, c="white", ls="--")
    ax.axvline(x=-1126+theta_c/4, c="white", ls="--")
    # fig.savefig(f'./img/Bruco{i}.png')
    #fig.savefig(f'./pdf/Bruco{i}.pdf')
    #plt.close(fig)
    plt.show()
    
    z=hh[0]
    x_list = hh[1]
    y_list =hh[2]

    x_space = [[x_list[i]]*z.shape[1] for i in range(0, z.shape[0])]
    y_space = [y_list[:-1]]*z.shape[0]
    
BrucoplotSDAQ= not(Brucoplot)
if BrucoplotSDAQ:
    print(len(theta_in_x))
    deflX = theta_out_x-theta_in_x
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 7)
    rot = xinfo[:,0]
    print(rot)
    #fig.suptitle(f'Bruco Plot [{x_cut_low:.4f}_{x_cut_high:.4f} um]', fontsize = 20)
    bin_step = 2 #urad
    
    hh = ax.hist2d((theta_in_x)[logi_cry],(deflX)[logi_cry], 
                    bins = [100, 150],
                    cmap = my_cmap,
                    range = [[-100,100],[-100,150]])#,
                    #norm=mpl.colors.LogNorm())

    X_prof, Y_prof, sigma_Y = sb.hist2dToProfile(hh, "mean")
    ax.errorbar(X_prof, Y_prof, sigma_Y, color = "red", lw =0.2)
    #ax.grid()
    theta_c=50 #urad


    ax.set_xlabel('Theta X in [$\mu$rad]')
    ax.set_ylabel('Defl X [$\mu$rad]')


    fig.colorbar(hh[3], ax = ax)
    #ax.axvline(x=-1126-theta_c/4, c="white", ls="--")
    #ax.axvline(x=-1126+theta_c/4, c="white", ls="--")
    # fig.savefig(f'./img/Bruco{i}.png')
    #fig.savefig(f'./pdf/Bruco{i}.pdf')
    #plt.close(fig)

    z=hh[0]
    x_list = hh[1]
    y_list =hh[2]

    x_space = [[x_list[i]]*z.shape[1] for i in range(0, z.shape[0])]
    y_space = [y_list[:-1]]*z.shape[0]
    plt.show()

2584


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[48.851765 48.851765 48.851765 ... 48.851765 48.851765 48.851765]


In [15]:
# U_0_111=19
# E=120e9
# theta_c = np.sqrt(2*U_0_111/E)*10**6
theta_c=50 #urad

subrange = [-theta_c/4, theta_c/4, -100, 150]  # angular phase space subrange in which to perform 1d analysis
bXSide =True
fitRange = [30, 120]  # gaussian fit range
fitCentre = 62  # gaussian fit approx. centre
fitSigma = 40  # gaussian fit approx. sigma
nSigmasIntegrate = 2  # number of gaussian sigmas over which to perform peak integral (i.e. efficiency computation)
bCompute = False
bPlot=True
bAverage=True
bFit = True

In [16]:
if bCompute: 
    
    xPlot, yPlot = projPhaseSpace((np.array(x_space), np.array(y_space), np.array(z)), subrange, "x" if bXSide else "y", bAverage)
    xPlot, yPlot=rebin(xPlot, yPlot, nBin=2)
    area = simps(yPlot, xPlot)
    yPlot=yPlot/area
    
    if bFit:
        xFit0 = np.where((xPlot >= fitRange[0]) & (xPlot <= fitRange[1]), xPlot, np.nan)
        yFit0 = np.where((xPlot >= fitRange[0]) & (xPlot <= fitRange[1]), yPlot, np.nan)
        xFit1, yFit1 = xFit0[np.logical_not(np.isnan(xFit0))], yFit0[np.logical_not(np.isnan(yFit0))]

        p0 = [max(yFit1), fitCentre, fitSigma]
        par, cov = curve_fit(sl.fGaus, xFit1, yFit1, p0=p0, maxfev = 800)  # fit output: par = (ampl, mean, sigma)
        xFit = np.linspace(min(xFit1), max(xFit1), 1000)
        yFit = sl.fGaus(xFit, *par)
        gaussIntegral = quad(lambda xx : sl.fGaus(xx, *par), par[1]-nSigmasIntegrate*par[2], par[1]+nSigmasIntegrate*par[2])

        print("gaussian peak HEIGHT is %f" % par[0])
        print("gaussian peak CENTRE is %f urad" % par[1])
        print("gaussian peak SIGMA is %f urad" % par[2])
        if bAverage:
            print("gaussian peak integral over total i.e. EFFICIENCY is %f" % (gaussIntegral[0]/1))
        else:
            print("toggle average over same-abscissa points to get a value for the EFFICIENCY")

    if bPlot:
        plt.close("channelingFits")
        plt.figure("channelingFits", figsize=[8, 5])
        plt.title("angular phase space projection to %s axis" % "y" if bXSide else "x")
        plt.xlabel("output angle [urad]")

        plt.plot(xPlot, yPlot, marker=".", ms=3, lw=0, c="C0")
        if bFit:
            plt.plot(xFit, yFit, c="r", lw=1)
            plt.axvline(x = par[1], c="r", lw=1)
            plt.axvline(x = par[1]-par[2], c="r", ls=":", lw=1)
            plt.axvline(x = par[1]+par[2], c="r", ls=":", lw=1)

        plt.tight_layout()
        plt.show()